In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer

In [2]:
train = pd.read_csv("train.csv", header=None )
test = pd.read_csv("test.csv", header=None)
trainLabels = pd.read_csv("trainLabels.csv", header =None)

In [3]:
print(f'Train shape: {train.shape}\nTest shape: {test.shape}\nTrainLabels:{trainLabels.shape}')

Train shape: (1000, 40)
Test shape: (9000, 40)
TrainLabels:(1000, 1)


In [4]:
X_train, X_val, y_train, y_val = train_test_split(train, trainLabels, test_size = 0.2, random_state = 5)

In [5]:
std = StandardScaler()
mms = MinMaxScaler()
nor = Normalizer()
train_std = std.fit_transform(train)
train_mms = mms.fit_transform(train)
train_nor = nor.fit_transform(train)

In [6]:
clf = GradientBoostingClassifier()
clf.fit(train, trainLabels)

D:\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [7]:
clf.score(X_val,y_val)

0.99

In [8]:
cross_val_score(clf, X_val,y_val, cv=5).mean()

D:\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y,

0.825

In [9]:
n_estimators = [50, 100, 150]
max_depth = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)
grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)
grid_result = grid_search.fit(train_std, trainLabels)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    5.7s finished
D:\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: -0.122000 using {'max_depth': 3, 'n_estimators': 150}


In [11]:
clf_bestparam = GradientBoostingClassifier(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
clf_bestparam.fit(train, trainLabels)

# 預測測試集
y_pred = clf_bestparam.predict(test)

D:\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:
result = pd.DataFrame(y_pred)
result.shape

(9000, 1)

In [14]:
result.columns = ['Solution']
result['Id'] = np.arange(1,result.shape[0]+1)

In [15]:
result = result[['Id','Solution']]

In [16]:
result.to_csv('submission.csv', index=False)